In [2]:
import pandas as pd
import numpy as np
import cv2
import argparse
import os
import sys
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint

In [3]:
# ---------------------------------------------------------------
# CONSTANTS RELATED TO VIDEO DATA
# ---------------------------------------------------------------

# directory where videos of different shots are stored
VIDEO_DIRECTORY = "videos"
# path to each one of the directories containing vidoes belonging to different shot types
CATEGORY_DIRECTORIES = glob(os.path.join(VIDEO_DIRECTORY, "*"))
# categories or shot types
CATEGORIES = [x.split(os.sep)[-1] for x in CATEGORY_DIRECTORIES]
# number of categories (also equal to the number of nuerons in the output layer)
NUM_CATEGORIES = len(CATEGORIES)
# training data is generated from the video clips and saved as a CSV file
TRAIN_CSV_FILENAME = "train_data.csv"
# testing data is generated from the video clips and saved as a CSV file
TEST_CSV_FILENAME = "test_data.csv"
IM_WIDTH = 240
IM_HEIGHT = 135
FRAME_RATE = 30

# ---------------------------------------------------------------
# CONSTANTS RELATED TO VIDEO DATA
# ---------------------------------------------------------------

# name of the model which saves the optimal weights after training
# 320 x 180 is chosen so that it has the same aspect ratio as the original vid (1920 x 1080)
MODEL_NAME = "weights.hdf5"

# Rename Videos

In [4]:
print("Renaming videos...")
for i in range(NUM_CATEGORIES):
    videos = glob(os.path.join(CATEGORY_DIRECTORIES[i], "*"))
    category = CATEGORIES[i]
    counter = 0
    for video in videos:
        counter += 1
        new_name = f"videos\\{category}\\{category}_{counter:05d}.mp4"
        os.rename(video, new_name)

Renaming videos...


# Convert videos to images

In [5]:
images = []
print("Converting videos to images...")
for i in range(NUM_CATEGORIES):
    videos = glob(os.path.join(CATEGORY_DIRECTORIES[i], "*"))
    counter = 0
    for video in videos:
        cap = cv2.VideoCapture(video)
        frames = 0
        # loop until there are no frames left or the number of frames exceeds FRAME_RATE (30)
        while True and frames < FRAME_RATE:
            frames += 1
            counter += 1
            ret, frame = cap.read()
            if (not ret):  # no more frames are left in the video
                break
            # create a directory called images if it doesn't already exist
            if not os.path.exists("images"):
                os.mkdir("images")
            img_dir = os.path.join(
                os.getcwd(), "images", CATEGORIES[i])
            # create sub-directory inside images with the label name if it doesn't already exist
            if not os.path.exists(img_dir):
                os.mkdir(img_dir)
            img_name = f"{CATEGORIES[i]}_{counter:05d}.jpg"
            img_path = os.path.join(img_dir, img_name)
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            resized_gray_frame = cv2.resize(
                gray_frame, (IM_WIDTH, IM_HEIGHT), interpolation=cv2.INTER_AREA)
            if not cv2.imwrite(img_path, resized_gray_frame):
                raise Exception("Failed to write image")
            images.append((img_path, CATEGORIES[i]))

Converting videos to images...


# Split into test and train set

In [6]:
print("Splitting images into training set and testing set...")
X = []
y = []
for (img_path, label) in images:
    X.append(img_path)
    y.append(label)
X = np.array(X)
y = np.array(y)
# the stratify parameter is used to make sure that the distribution of each class is similar in test and train sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y)

Splitting images into training set and testing set...


# Preprocess images

In [7]:
image_loader = lambda x : image.load_img(x, target_size=(IM_WIDTH, IM_HEIGHT, 3))
image_to_array = lambda x : image.img_to_array(x)
image_normalize = lambda x : x / 255.0

temp_train = []
temp_test = []

for img in X_train:
    img = image_loader(img)
    img = image_to_array(img)
    img = image_normalize(img)
    temp_train.append(img)

for img in X_test:
    img = image_loader(img)
    img = image_to_array(img)
    img = image_normalize(img)
    temp_test.append(img)
    
X_train = np.array(temp_train)
X_test = np.array(temp_test)

X_train.shape

(2889, 240, 135, 3)

In [8]:
le = LabelEncoder()
le.fit(CATEGORIES)

y_test_lab = le.transform(y_test)
y_train_lab = le.transform(y_train)

In [9]:
base_model = VGG16(weights="imagenet", include_top=False)

X_train = base_model.predict(X_train, batch_size=16)
X_test = base_model.predict(X_test, batch_size=16)

In [10]:
X_train.shape

(2889, 7, 4, 512)

In [14]:
X_train = X_train.reshape(
    X_train.shape[0], X_train.shape[1] * X_train.shape[2] * X_train.shape[3])
X_test = X_test.reshape(
    X_test.shape[0], X_test.shape[1] * X_test.shape[2] * X_test.shape[3])

maxval = X_train.max()
X_train = X_train / maxval
X_test = X_test / maxval

IndexError: tuple index out of range

In [25]:
X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_train.shape

(2889, 14336)

In [26]:
model = Sequential()
model.add(LSTM(256,dropout=0.2, batch_input_shape=(16, 1, X_train.shape[1]), return_sequences=True))
model.add(Dense(1024, activation="relu", input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(NUM_CATEGORIES, activation="softmax"))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (16, 1, 256)              14943232  
                                                                 
 dense_15 (Dense)            (16, 1, 1024)             263168    
                                                                 
 dropout_12 (Dropout)        (16, 1, 1024)             0         
                                                                 
 dense_16 (Dense)            (16, 1, 512)              524800    
                                                                 
 dropout_13 (Dropout)        (16, 1, 512)              0         
                                                                 
 dense_17 (Dense)            (16, 1, 256)              131328    
                                                                 
 dropout_14 (Dropout)        (16, 1, 256)             

In [27]:
checkpoint = ModelCheckpoint(MODEL_NAME, save_best_only=True, monitor="val_loss", mode="min")
model.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=200, validation_data=(
    X_test, y_test), callbacks=[checkpoint], batch_size=128)

Epoch 1/200


ValueError: in user code:

    File "C:\Users\saile\Documents\Sailesh\Programming\Projects\SportsTrack\Application\MachineLearning\ShotClassification\venv\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\saile\Documents\Sailesh\Programming\Projects\SportsTrack\Application\MachineLearning\ShotClassification\venv\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\saile\Documents\Sailesh\Programming\Projects\SportsTrack\Application\MachineLearning\ShotClassification\venv\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\saile\Documents\Sailesh\Programming\Projects\SportsTrack\Application\MachineLearning\ShotClassification\venv\lib\site-packages\keras\engine\training.py", line 859, in train_step
        